# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [18]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [19]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [20]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'local', 'mynewDB', 'traveldb', 'uk_food']


In [21]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [22]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [23]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64fa03619c2fa664b0201ff7'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [24]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [25]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {}

In [26]:
# Insert the new restaurant into the collection
establishments.insert_one({'BusinessName': 'Penang Flavours'})

In [27]:
# Check that the new restaurant was inserted
establishments.find_one({'BusinessName': 'Penang Flavours'})

{'_id': ObjectId('64fa04f25f9efcb2bd2814ee'),
 'BusinessName': 'Penang Flavours'}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [28]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
q = {'BusinessType': 'Restaurant/Cafe/Canteen'}
f = ['BusinessTypeID', 'BusinessType']

establishments.find_one(q, f)

{'_id': ObjectId('64fa03619c2fa664b0201ff7'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [29]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    new_restaurant_data, 
    {'$set': 
        {'BusinessTypeID': 1}
    }
)

In [30]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName':'Penang Flavours'})

{'_id': ObjectId('64fa04f25f9efcb2bd2814ee'),
 'BusinessName': 'Penang Flavours'}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [31]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName': 'Dover'})

994

In [32]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

In [33]:
# Check if any remaining documents include Dover
pprint(establishments.count_documents({'LocalAuthorityName': 'Dover'}))

0


In [34]:
# Check that other documents remain with 'find_one'
establishments.find_one({})

{'_id': ObjectId('64fa03619c2fa664b02022de'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [35]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}, 
                                         'geocode.latitude': {'$toDouble': '$geocode.latitude'}
                                         }
                                }
                               ]
                          )

Use `update_many` to convert `RatingValue` to integer numbers.

In [51]:
# Set non 1-5 Rating Values to Null
null_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [54]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])

In [55]:
# Check that the coordinates and rating value are now numbers
[doc for doc in establishments.find({}, ['geocode.longitude', 'geocode.latitude'])]

[{'_id': ObjectId('64fa03619c2fa664b02022de'),
  'geocode': {'longitude': 1.195625, 'latitude': 51.083812}},
 {'_id': ObjectId('64fa03619c2fa664b02022df'),
  'geocode': {'longitude': 1.196408, 'latitude': 51.086058}},
 {'_id': ObjectId('64fa03619c2fa664b02022e0'),
  'geocode': {'longitude': 1.194762, 'latitude': 51.085797}},
 {'_id': ObjectId('64fa03619c2fa664b02022e3'),
  'geocode': {'longitude': 1.188537, 'latitude': 51.08084}},
 {'_id': ObjectId('64fa03619c2fa664b02022e4'),
  'geocode': {'longitude': 1.188537, 'latitude': 51.08084}},
 {'_id': ObjectId('64fa03619c2fa664b02022e5'),
  'geocode': {'longitude': 1.188537, 'latitude': 51.08084}},
 {'_id': ObjectId('64fa03619c2fa664b02022e6'),
  'geocode': {'longitude': 1.188537, 'latitude': 51.08084}},
 {'_id': ObjectId('64fa03619c2fa664b02022e7'),
  'geocode': {'longitude': 1.18590330311705, 'latitude': 51.0783519967076}},
 {'_id': ObjectId('64fa03619c2fa664b02022e8'),
  'geocode': {'longitude': 1.18590330311705, 'latitude': 51.0783519967